# Deep Learning & Artificial Intelligence
## Neural Networks Foundations, Chapter 2
### Dr. Jie Tao, Fairfield University

## Math behind Neural Networks

Understanding how neural networks work often requires in-depth understanding of several mathematical concepts, which includes but not limited to:
- tensors and their operations
- differentiation
- gradient descent
- derivatives
- computation graph
- vectors and matrix

Additionally, you also need to know the basics of:
- (logistic) regression
- (binary) classification
- Broadcasting in Python, and basics of NumPy

Which you already learned in your previous courses.

### What are Tensors?

You must remember N-dimensional arrays in `NumPy`, as a matter of fact that is the __most popular__ data type we use in Deep Learning, and all mainstream machine learning models.

**Fun fact**: TensorFlow gets its name because of tensors.

But what are tensors?
- Tensors are containers for data
  - Almost always __numerical__ data
- If you know matrix:
  - A matrix is a `2D` tensor
- So tensors are implementations of matrices
  - With an arbitrary number of dimensions (e.g., `2`)

### What are Scalars?

Any number in a tensor is called a _scalar_.
- Scalar, aka. scalar tensor, is _0-dimensional_
- A `NumPy` `float32` or `float64` number is a __scalar__
  - remember you can display the dimensions of a ``NumPy`` array using the `ndim` attribute?
  - For a scalar, `ndim == 0`

In [ ]:
import numpy as np

x = np.array(42)
assert x.ndim == 0

### What are vectors?

An array of numbers is called a _vector_.

- A vector is 1-dimensional tensor
- Note that a vector is usually a ``NumPy`` 1D array, not a __list__
  - It's because arrays allow us to contain __different__ types on data in them

In [ ]:
vec = np.array([1, 2, 3])
assert vec.ndim == 1

### High-dimensional Tensors

- By default, a tensor has a ``ndim >= 2``
  - If it's a 2D tensor, it is a regular matrix
  - If it's more than 2D, then it's called a _high-dimensional_ tensor
  - For instance, usually we represent _one_ image as a 2D or 3D tensor:
    - If it's 2D --> represent pixels
    - If it's 3D --> represent pixels and the color channel
  - In that sense, if you have _multiple_ images, then it will be a 3D/4D image
    - The additional dimension represents the collection of all images

In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images.shape

11493376/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

Look at the results above:
- `60000` means we have 60,000 images in the _training_ dataset
- each image has `28` pixels along the horizontal and vertical dimensions, respectively.

### Key Attributes of Tensors

A tensor has three key features:
- Rank: number of _main_ dimensions. We use _rank_ to determine the size of a tensor. For instance, for a `2D` tensor, rank == 2. We often use `ndim` for that;
- Shape: show __all__ the dimensionalities of the tensor. We often use ``.shape`` for that.
  - For instance, a regular `3D` tensor should show something like ``(60000, 28, 28)``.
  - If you see something like ``(60000,)``, that means either this is a `1D` tensor, or the shapes in the 2D tensors (in each of the 60,000 are not uniform
- Data Type: type of scalars in a tensor. We can use `dtype` to retrive them. Like said above, usually these are `float32` or `float64`.

In [ ]:
# getting the rank of a tensor - 3D tensor so rank == 3
train_images.ndim

3

In [ ]:
# Shape of tensor
train_images.shape

(60000, 28, 28)

In [ ]:
# data type - in this case they are integers
train_images.dtype

dtype('uint8')

### Real World Use of Tensors

We typically map real world data into tensors like below:

- Tabular data - `2D` tensors with shape as ``(sampels, features)``
- Time-series or other sequential data - `3D` tensors with shape as ``(samples, timestep, features)``
- Image data: `4D` tensors with shape as ``(samples, height, weight, channels)`` or ``(samples, channels, height, weight)``
  - in the `mnist` example, since we are dealing with _grayscale_ data, `channel == 1` and is omitted
- Video data: `5D` tensors with shape as ``(samples, frames, height, weight, channels)`` or ``(samples, frames, channels, height, weight)``
  - Since videos are sequence of images, video data are combination of `image data` and `sequence data`.
  - In that case `frames` are the same as `timesteps`

Refer to your textbook for more details.

### Manipulating Tensors in NumPy

If you are using `Keras` or `TensorFlow` for deep learning, we usually prepare our data in `NumPy`.

Fun fact: if you are using `PyTorch` then it's 50% `NumPy` and 50% `PyTorch`.

The most important operation here is __slicing__. Refer to the [NumPy Official Docs](https://numpy.org/doc/stable/reference/arrays.indexing.html) if you need a refresher.



In [ ]:
data_slice = train_images[:10]
data_slice.shape

(10, 28, 28)

### Tensors and NumPy arrays

You may have already discovered that tensors and `NumPy` arrays are very similar. In real world, we do use them interchangeably.

In [ ]:
import torch
import numpy as np
array_a = np.arange(6).astype(float)
array_a, type(array_a)

(array([0., 1., 2., 3., 4., 5.]), numpy.ndarray)

In [ ]:
### array to tensor
tensor_a = torch.tensor(array_a)
tensor_a, type(tensor_a)

(tensor([0., 1., 2., 3., 4., 5.], dtype=torch.float64), torch.Tensor)

In [ ]:
### convert back to array
array_a_rep = tensor_a.numpy()
assert np.array_equal(array_a_rep, array_a) ### equal to the original array

We are going to talk more about tensor operations in Workshop 1. For now, let's just assume `NumPy` arrays are tensors, that is particularly true in `tensorflow`/`keras`.

### The Notion of Batch

Slicing is very important since we use that to create _batches_. In order to introduce the notion of batch, we need to review what is a _sample_:
- A sample is a row in your data, aka an instance. For intance in `train_images` we have `60000` samples.
- A batch is a slice of samples. In deep learning, we update the model parameters after walking through a batch.
  - `batch_size` is an important hyperparameter that we use to tune our networks
  - At the end of each batch, the model makes predictions and the _error_ is computed
- A training dataset can be divided into one or more batchs, we will learn how it's divided when we discuss the idea of __gradient descent__.

__PRO TIP__: even though you can set `batch_size` arbitrarily, we often use $2^n$ for that, so typical values are ``16, 32, 128, 256, ...`` And it dependes on your data size.

### Tensor Operations

Think of a simple Single Layered Perceptron (SLP) ``dense`` layer in ``keras``, which can be implemented via the code below:

``keras.layers.Dense(512, activation='relu')``

This layer can be simplified as a mathematical function: if the input is a ``2D`` tensor then the output is a ``2D`` tensor. We can define the function as follows (where $X$ is a ``2D`` input, $W$ is the weights on $X$, and $b$ is a vector):

$$ y = relu((W \cdot X) + b) $$

Without the `relu` activation, you can see ``dense`` is simple a linear combination - what we saw a lot in regression models.

What are the operations above:
- $ \cdot $ (dot) this is a [dot product](https://en.wikipedia.org/wiki/Dot_product) of $W$ and $X$
- $ + $ just a simple addition, we need to make sure if $X$ is ``2D`` and $b$ is ``2D`` as well
- ``relu`` a simple operation: `relu(x) == max(x, 0)`.

$\cdot$ is a tensor operation.

![dot product](https://drek4537l1klr.cloudfront.net/chollet/Figures/02fig05.jpg  "Illustration of Dot Product")

### Element-wise Operations

Previous $+$ and `relu` operations are _element-wise_ operations: meaning the operation is applied to each scalar in a tensor. You can think of element-wise operations implemented using ``for`` loops in Python.

For instance, the `relu` operation can be implemented below (you can find the implementation of addition in the textbook):

In [ ]:
def naive_relu(x):
    assert len(x.shape) == 2

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x


With the help of ``NumPy`` (since ``NumPy`` automatically __broadcast__ any operation to the array), we can implement the above SLP layer without a ``for`` loop.

``y = np.max(np.dot(W, X) + b)``

### Tensor Reshaping

In order to perform tensor operations, the two tensors should be of the __same__ shape. If the two tensors are of _different_ shapes, we can always use `.reshape` to make them into the same.

In [ ]:
x = np.array([[0., 1.],
                 [2., 3.],
                 [4., 5.]])
x.shape


(3, 2)

In [ ]:
x = x.reshape((6, 1))
x.shape


(6, 1)

In [ ]:
x = x.reshape((2, 3))
x

array([[0., 1., 2.],
       [3., 4., 5.]])

A specific type of reshape is transpose, which means change the original first dimension ("rows") into second dimension ("columns"), and vise versa.

## Gradient Descent - Optimizing the NN

Gradient Descent is the key to optimize any NN - optimizing in this context means finding the __best__ _model parameters_ in training.

Let's start by reviewing the SLP layer we saw before:

$$ \hat{y} = relu((W \cdot X) + b) $$

$W$ and $b$ above are the _model parameters_ in the layer. The objective is to find the __best__ values for $W$ and $b$ so $\hat{y}$ is close to $y$. $W$ and $b$ contains information learned from the data.

### Training a NN

0. Fill $W$ and $b$ with _small, random_ values - no need to calculate $\hat{y}$ since $W$ and $b$ are random;
1. Sample a __batch__ of $X$ and $y$ - these batches are ordered, refer to [here](#scrollTo=1BTOFabbs1F5&line=10&uniqifier=1) for the notion of _batch_;
2. Calculate $\hat{y}$ based on $X$ - this step is called __forward pass__;
3. Compute the loss on the batch - which measures the difference between $y$ and $\hat{y}$ - aka. _error_ in ML;
4. Update $W$ and $b$ so the _loss_ decreases;
5. Repeat steps 1 - 4 until loss no longer decreases.

### Loss Function

Loss function is one of the keys in training NNs. A lot of the evaluation metrics, such as `MAE` or `classification accuracy`, can be used as loss functions.

However, in NNs we use the __Nagative Log-Likelihood__ (NLL) as the main loss function. Refer to [this article](https://towardsdatascience.com/log-loss-function-math-explained-5b83cd8d9c83) for more information. We will talk about loss functions next week.

### Challenges in Training NN

Calculating loss is straightforward - just simple tensor operations as we seen before. The key challenge is:



> How do you determine whether we should _increase_ or _decrease_ $W$ and $b$, and by how much?

Inituitively we can borrow the idea of hyperparameter tuning from ML: freezing all other weights in a NN, just change one scalar value at a time, see how the loss changes. If the loss decreases, moving on to the next. But this is __very inefficient__!!

The solution is called __gradient__, since the operations in NNs are __differentiable__. Below is a refresher on these concepts.



### What is a derivative?

Given a _continuous, smooth_ function $ y = f(x)$, mapping a real number $x$ to another real number $y$.

Since the function is continuous, a small change in $x$ ($\epsilon_{x}$) will lead to a small change in $y$ ($\epsilon_{y}$):

$$ y + \epsilon_{y} = f(x + \epsilon_{x}) $$

Additionally, since the function is _smooth_ (non-linear), if $\epsilon_{x}$ is small enough, at a certain point $p$, we can use a linear function with a slope $\alpha$, so that $ \epsilon_{y} = \alpha \times \epsilon_{x} $:

$$ y + \alpha \times \epsilon_{x} = f(x + \epsilon_{x}) $$

In which, $\alpha$ is called a derivative of $f$ at $p$. If $\alpha$ is positive, $y$ increases when $x$ increases, and vice versa. And the absolute value of $\alpha$ (called __magnitude__) measures how fast $y$ changes with $x$. Sounds familiar? This is the same as the coefficient in any linear model.

### Illustration of Derivatives

![derivative](https://drek4537l1klr.cloudfront.net/chollet/Figures/02fig10.jpg)

For $f(x)$ to be __differentiable__ (means "having derivatives"), $f(x)$ needs to be __continuous__ and __smooth__. For all these $f(x)$, these exists a _derivative_ function $f'(x)$ that maps values of $x$ to the slope of the local linear approximation of $f$ in those points.

For instance, if $f(x) = cos(x)$, $f'(x) = - sin(x)$. If $f(x) = \alpha \times x + b $, $f'(x) = \alpha $.

With the help of $f'(x)$, we can control how to move $f(x)$. Suppose we want to decrease $f(x)$ and if $f'(x) > 0$, we need to make $\epsilon_{x} < 0$; and vice versa.

### Gradients: the Derivative of Tensor Operations

Before we were dealing with derivatives of _scalar_ values, if we talk about _multi-dimensional_ inputs (tensors), the derivatives become a gradient.

If $\hat{y} = f(X) = W \cdot X $, the loss value is $ loss(\hat{y}, y) $, then we can have a loss function $ loss(\hat{y}, y) = g(W) $. For any value of $ W_0 \in W $, $g(W_0)$ has the same shape of $W$.

If $ f(x) $ is of a scalar value (as we seen before), $f'(x)$ is the _slope_ of the $ f(x) $ curve. Similarly, $g(W_0)$ is the __curvature__ of $g(W)$.

Keep in mind the training objective is to _minimize_ $ loss(\hat{y}, y) $, which means is to minimize $ g(W) $. Say we start with $ W_0 $, the next point should be $ W_1 = W_0 - step \times g(W_0) $, if $ g(W_0) > 0$; so we get $ g(W_1) < g(W_0) $. By repeating this step, we can ultimately get $ g(W_n) = min(g(W)) $.

Note that $step$ is a required hyperparameter here, later we will refer to this as __learning rate__.
- Typically __learning rate__ is positive;
- Also __learning rate__ should be small, if it's too big we may skip the _minimal_ value;
- Of course if it's too small the training is not efficient.

### Stochastic gradient descent

Theoretically (or mathematically), for any function $ g(W)$, at the minimal point its derivative is `0`. So to minimize the loss function, we just need to find all points where $ g(W) = 0 $. This is a polynomial function of $N$ variables, where $N$ is the number of neurons in a layer. If $N$ is small, we can easily solve it. But in a NN we are talking about $N$ of thousands, if not millions, so we need an algorithm to solve it.

0. Fill all parameters with _small, random_ values
1. Draw a batch of training samples $X$ and corresponding targets $y$.
3. Run the network on $X$ to obtain predictions $ \hat{y} $.
4. Compute the loss of the network on the batch $loss(\hat{y}, y)$.
5. Compute the gradient of the loss with regard to the network’s parameters (a **backward pass**).
6. Move the parameters a little in the opposite direction from the gradient — for example $W = step \times gradient$ — thus reducing the loss on the batch a bit.
7. Repeating steps 1 - 6 until the gradient is `0`.

This is called a _mini-batch Stochastic Gradient Descent_ (mini-batch SGD) algorithm. Note that if the _batch_ is `1`, then it is a _true_ SGD algorithm. If the _batch_ size equals to the training sample size, it's a _batch_ SGD algorithm.

### Illustration of SGD

If there is only 1 parameter in a NN:

![1D-SGD](https://drek4537l1klr.cloudfront.net/chollet/Figures/02fig11.jpg)

If there are 2 parameters:

![2D-SGD](https://drek4537l1klr.cloudfront.net/chollet/Figures/02fig12.jpg)



### Variants of SGD - Optimizers of NN

Researchers have suggested looking into the previous changes in the weights, rather than just the current gradients. This is called SGD with momentum. Examples include `Adagrad`, `RMSProp`, and so on.

These are known as optimize of NN, which we will learn in the next week.

SGD with momentum solves two problems:
1. **local minima**: We need to find the __global minima__ (the absolute minimum) of the loss function - thus the learning rate cannot be too big;
2. __convergence speed__: if the learning rate is too small, the convergence speed is low - meaning training will take a lot of time.

![local minima](https://drek4537l1klr.cloudfront.net/chollet/Figures/02fig13.jpg)

### Backpropgation - the Latest Breakthrough of NN

Recall that we define any _continuous, smooth_ function $f(x)$ is __differentiable__, we can explicitly find the derivative $f'(x)$. However, in deep learning, because of the multiple layers, the gradients are _chained_: meaning the gradient from the current layer is linked with all previous layers.

For instance, let's assume a NN ($F$) with 2 layers (with 2 tensor operations $f$, $g$, and weights as $W_a$, $W_b$):

$$ F(W_b, W_a) = g(W_b, f(W_a)) $$

Based on the [chain rule](https://en.wikipedia.org/wiki/Chain_rule), we know that:

$$ [F(X)]' = f'(g(X)) \times g'(X) $$

Based on this, we have __backpropgation__, which starts with the _final_ loss value, and works backwards from the last (next to *output*) layer to the first (second to *input*) layer. In each step of the chain, the goal is to _minimize_ the current loss value.

Thanks to __symbolic differentiation__, modern packages like `TensorFlow` can compute the gradient __function__ for the whole chain - which means you do not have to implement _backpropgation_ on your own. But if you want to learn more about this, refer to [this wonderful presentation](https://cs230.stanford.edu/files/C1M2.pdf) by one of the giants (Andrew Ng) in the field.


# Deep Learning & Artificial Intelligence
## Neural Networks Foundations, Chapter 2
### Dr. Jie Tao, Fairfield University